In [1]:
import os
import subprocess
import requests
import tempfile
from rdkit import Chem
from rdkit.Chem import AllChem
from biopandas.pdb import PandasPdb

In [2]:
def download_pdb(pdb_id, output_dir="."):
    """Download PDB file from RCSB database using updated API"""
    pdb_id = pdb_id.upper()
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    output_path = os.path.join(output_dir, f"{pdb_id}.pdb")
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        with open(output_path, 'wb') as f:
            f.write(response.content)
        return output_path
    except Exception as e:
        print(f"Error downloading PDB file: {e}")
        return None

In [3]:
def smiles_to_pdbqt(smiles, output_dir="."):
    """Convert SMILES to PDBQT using Open Babel 3.1"""
    try:
        # Create RDKit molecule and generate 3D coordinates
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES string")
        
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol)
        
        # Save as SDF file (better handling of 3D coordinates)
        with tempfile.NamedTemporaryFile(suffix='.sdf', delete=False) as tmp:
            tmp_path = tmp.name
            writer = Chem.SDWriter(tmp_path)
            writer.write(mol)
            writer.close()
        
        # Convert to PDBQT using Open Babel 3.1
        pdbqt_path = os.path.join(output_dir, "ligand.pdbqt")
        cmd = [
            "obabel",
            tmp_path,
            "-O", pdbqt_path,
            "--gen3d",  # Ensure 3D coordinates if needed
            "--partialcharge", "gasteiger",
            "--title", "LIGAND"  # Add title for tracking
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode != 0:
            raise RuntimeError(f"Open Babel failed: {result.stderr}")
        
        os.unlink(tmp_path)  # Clean up temporary file
        
        return pdbqt_path
    except Exception as e:
        print(f"Error converting SMILES to PDBQT: {e}")
        if os.path.exists(tmp_path):
            os.unlink(tmp_path)
        return None


In [4]:

def prepare_receptor(pdb_path, output_dir="."):
    """Prepare receptor using updated methodology"""
    try:
        # Read and clean PDB
        ppdb = PandasPdb()
        ppdb.read_pdb(pdb_path)
        
        # Remove waters and unwanted heteroatoms
        clean_df = ppdb.df['HETATM'][~ppdb.df['HETATM']['residue_name'].isin(['HOH', 'WAT'])]
        ppdb.df['HETATM'] = clean_df
        
        # Save cleaned PDB
        cleaned_pdb = os.path.join(output_dir, "receptor_clean.pdb")
        ppdb.to_pdb(path=cleaned_pdb, records=['ATOM', 'HETATM'])
        
        # Convert to PDBQT using Open Babel 3.1 (alternative to MGLTools)
        pdbqt_path = os.path.join(output_dir, "receptor.pdbqt")
        cmd = [
            "obabel",
            cleaned_pdb,
            "-O", pdbqt_path,
            "-x", "r",  # Treat as rigid molecule
            "--partialcharge", "gasteiger"
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode != 0:
            raise RuntimeError(f"Receptor preparation failed: {result.stderr}")
        
        return pdbqt_path
    except Exception as e:
        print(f"Error preparing receptor: {e}")
        return None

In [5]:

def calculate_binding_box(pdbqt_path, padding=8):
    """Calculate binding box with improved coordinate handling"""
    try:
        # Read PDBQT file
        with open(pdbqt_path, 'r') as f:
            lines = [line for line in f if line.startswith(('ATOM', 'HETATM'))]
        
        if not lines:
            raise ValueError("No valid atoms found in receptor file")
        
        # Extract coordinates more robustly
        coords = []
        for line in lines:
            try:
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                coords.append((x, y, z))
            except ValueError:
                continue
        
        if not coords:
            raise ValueError("No valid coordinates found")
        
        # Calculate box dimensions
        min_coords = [min(c[i] for c in coords) for i in range(3)]
        max_coords = [max(c[i] for c in coords) for i in range(3)]
        
        center = [(min_coords[i] + max_coords[i])/2 for i in range(3)]
        size = [max_coords[i] - min_coords[i] + 2*padding for i in range(3)]
        
        return {
            'center_x': center[0],
            'center_y': center[1],
            'center_z': center[2],
            'size_x': size[0],
            'size_y': size[1],
            'size_z': size[2]
        }
    except Exception as e:
        print(f"Error calculating binding box: {e}")
        return None


In [6]:


def run_docking(receptor_pdbqt, ligand_pdbqt, box_params, output_dir=".", exhaustiveness=8):
    """Run docking with improved error handling"""
    try:
        output_path = os.path.join(output_dir, "docking_results.pdbqt")
        
        cmd = [
            "vina",
            "--receptor", receptor_pdbqt,
            "--ligand", ligand_pdbqt,
            "--center_x", f"{box_params['center_x']:.3f}",
            "--center_y", f"{box_params['center_y']:.3f}",
            "--center_z", f"{box_params['center_z']:.3f}",
            "--size_x", f"{box_params['size_x']:.3f}",
            "--size_y", f"{box_params['size_y']:.3f}",
            "--size_z", f"{box_params['size_z']:.3f}",
            "--exhaustiveness", str(exhaustiveness),
            "--out", output_path,
            "--log", os.path.join(output_dir, "vina_log.txt")
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # Parse results more robustly
        scores = []
        if os.path.exists(output_path):
            with open(output_path, 'r') as f:
                for line in f:
                    if "REMARK VINA RESULT" in line:
                        try:
                            scores.append(float(line.split()[3]))
                        except (IndexError, ValueError):
                            continue
        
        return {
            'output_path': output_path,
            'scores': scores,
            'log': result.stdout
        }
    except Exception as e:
        print(f"Error running docking: {e}")
        return None


In [7]:

def visualize_in_pymol(receptor_pdb, docking_results_path):
    """Enhanced PyMOL visualization with modern features"""
    try:
        import pymol
        from pymol import cmd, util
        
        # Initialize PyMOL
        pymol.finish_launching(['pymol', '-q'])
        
        # Load receptor
        cmd.load(receptor_pdb, 'receptor')
        cmd.show('cartoon', 'receptor')
        cmd.color('lightblue', 'receptor')
        
        # Load docking results
        cmd.load(docking_results_path, 'docking_results')
        
        # Create object for best pose
        cmd.create('ligand', 'docking_results and state 1')
        cmd.delete('docking_results')
        
        # Style ligand
        cmd.show('sticks', 'ligand')
      #  util.cbag('ligand')print("\n=== DOCKING RESULTS ===")
        cmd.set('stick_radius', 0.15)
        
        # Show binding site residues
        cmd.select('binding_site', 'byres ligand around 5')
        cmd.show('sticks', 'binding_site')
        cmd.color('yellow', 'binding_site')
        
        # Show interactions
        cmd.distance('hbonds', 'receptor', 'ligand', mode=2, cutoff=3.5)
        cmd.set('dash_gap', 0.5)
        cmd.set('dash_color', 'red')
        
        # Center and zoom
        cmd.orient()
        cmd.zoom('ligand', 8)
        
        # Add informative labels
        cmd.label('binding_site and name CA', '"%s" % (resn+resi)')
        cmd.set('label_size', 14)
        cmd.set('label_color', 'black')
        
        print("\nPyMOL visualization launched showing the best binding pose.")
    except Exception as e:
        print(f"Error in PyMOL visualization: {e}")
        print("Ensure PyMOL is installed and available in your Python environment.")


In [8]:

def main():
    # Create output directory with timestamp
    from datetime import datetime
    output_dir = f"docking_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Get user input
    print("\n=== Molecular Docking Workflow ===")
    smiles = input("Enter ligand SMILES string: ").strip()
    pdb_id = input("Enter receptor PDB ID (4 characters): ").strip().upper()
    
    # Validate input
    if not smiles or not pdb_id or len(pdb_id) != 4:
        print("Invalid input. Please provide valid SMILES and 4-character PDB ID.")
        return
    
    print("\nStarting docking workflow...")
    
    # Step 1: Download receptor
    print("\n1. Downloading receptor...")
    pdb_path = download_pdb(pdb_id, output_dir)
    if not pdb_path:
        return
    
    # Step 2: Prepare ligand
    print("\n2. Preparing ligand...")
    ligand_pdbqt = smiles_to_pdbqt(smiles, output_dir)
    if not ligand_pdbqt:
        return

    print (ligand_pdbqt)
    
    # Step 3: Prepare receptor
    print("\n3. Preparing receptor...")
    receptor_pdbqt = prepare_receptor(pdb_path, output_dir)
    if not receptor_pdbqt:
        return

    print (receptor_pdbqt)
    
    # Step 4: Calculate binding box
    print("\n4. Calculating binding box...")
    box_params = calculate_binding_box(receptor_pdbqt)
    if not box_params:
        return
    print(box_params)
    
    # Step 5: Run docking
    print("\n5. Running docking...")
    docking_results = run_docking(receptor_pdbqt, ligand_pdbqt, box_params, output_dir)
    if not docking_results or not docking_results['scores']:
        print("Docking failed or produced no valid results")
        return
    
    # Display results
    print("\n=== DOCKING RESULTS ===")
    print(f"\nBest docking score: {docking_results['scores'][0]:.2f} kcal/mol")
    print(f"Number of poses: {len(docking_results['scores'])}")
    
    # Save detailed report
    report_path = os.path.join(output_dir, "docking_report.txt")
    with open(report_path, 'w') as f:
        f.write("=== DOCKING REPORT ===\n\n")
        f.write(f"Receptor: {pdb_id}\n")
        f.write(f"Ligand SMILES: {smiles}\n\n")
        f.write(f"Best score: {docking_results['scores'][0]:.2f} kcal/mol\n")
        f.write(f"All scores: {', '.join(f'{s:.2f}' for s in docking_results['scores'])}\n\n")
        f.write("Docking log:\n")
        f.write(docking_results['log'])
    
    print(f"\nDetailed report saved to {report_path}")
    
    # Step 6: Visualize in PyMOL
    print("\n6. Launching visualization...")
    receptor_clean = os.path.join(output_dir, "receptor_clean.pdb")
    visualize_in_pymol(receptor_clean, docking_results['output_path'])
    
    print("\nDocking workflow completed successfully!")

if __name__ == "__main__":
    main()


=== Molecular Docking Workflow ===


Enter ligand SMILES string:  CCO
Enter receptor PDB ID (4 characters):  6HUP



Starting docking workflow...

1. Downloading receptor...

2. Preparing ligand...
docking_results_20250420_173849/ligand.pdbqt

3. Preparing receptor...
docking_results_20250420_173849/receptor.pdbqt

4. Calculating binding box...
{'center_x': 135.26500000000001, 'center_y': 125.29599999999999, 'center_z': 135.0985, 'size_x': 103.16600000000001, 'size_y': 127.24999999999999, 'size_z': 139.697}

5. Running docking...
Docking failed or produced no valid results
